# Magma Keras to TensorRT

### Importing

In [8]:
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.optimizers import Adam
from keras.models import Sequential,load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

import tensorflow as tf
import tensorflow.contrib.tensorrt as trt
from tensorflow.python.platform import gfile


import time 
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import os
from os.path import isfile, exists, isdir, join

import shutil

### For Jetson TX2

In [3]:
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

tf.keras.backend.set_learning_phase(0)

In [4]:
CNN_MODEL = 'MagmaCnnClassifier.hdf5'
TARGET_SHAPE = 8
DATA_SHAPE = (100,100,3)
OPTIMIZER = Adam()
BATCH_SIZE = 256
EPOCH = 2

SAVED_MODEL_DIR = './saved_model/'
MODEL_NAME = CNN_MODEL

DATA_SHAPE = (100,100,3)
TRAIN_DIR = "./data/train"
TEST_DIR = "./data/test"
RESULT_PREDICTION_CALLBACK = None

# Data preprocessing
### create predict_dir by move random pictures from test_dir  
#### remove and copy test_dir

In [5]:
TEST_COPY_DIR = './data/test_dir'
PREDICT_DIR = './data/predict_dir'

In [6]:
# delete folder if exist
if exists(TEST_COPY_DIR) and isdir(TEST_COPY_DIR):
    shutil.rmtree(TEST_COPY_DIR)
print('remove if exist test_dir success')

if exists(PREDICT_DIR) and isdir(PREDICT_DIR):
    shutil.rmtree(PREDICT_DIR)
print('remove if exist predict_dir success')

#copy test as test_dir, there are result as list of copy files
# from distutils.dir_util import copy_tree
# copy_tree('./data/test','./data/test_dir')
from subprocess import call
call(['cp','-a', TEST_DIR, TEST_COPY_DIR])
print('copy test to test_dir success')

remove if exist test_dir success
remove if exist predict_dir success
copy test to test_dir success


#### create predict_dir and random moving images

In [6]:
#random select images
CATEGORIES = ['0','1','2','3','4','5','6','7']
IMAGES_PER_FOLDER = 1

In [10]:
import random
for category in CATEGORIES:
    
    path_ct = join(TEST_COPY_DIR,category)
    path_pd = join(PREDICT_DIR, category)
    
    if not exists(path_pd):
        os.makedirs(path_pd)
    
    image_list = os.listdir(path_ct)
    random.shuffle(image_list)
    
    for img in image_list[:IMAGES_PER_FOLDER]:
        path_src = join(path_ct,img)
        path_des = join(path_pd,img)
        shutil.move(path_src, path_des)
    
    print('copy - category:',category, image_list[:IMAGES_PER_FOLDER])
    
    

copy - category: 0 ['1202.png']
copy - category: 1 ['5573.png']
copy - category: 2 ['3077.png']
copy - category: 3 ['8448.png']
copy - category: 4 ['8580.png']
copy - category: 5 ['1786.png']
copy - category: 6 ['767.png']
copy - category: 7 ['157.png']


# WorkFlow

### Keras to TensorRT
![alt text](pictures/Keras_to_TensorRT.png)

###### ----------------------------------------example model from medium

In [7]:
# https://medium.com/@pipidog/how-to-convert-your-keras-models-to-tensorflow-e471400b886a
# 1) build keras model + train
x = np.vstack((np.random.rand(1000,10),-np.random.rand(1000,10)))
y = np.vstack((np.ones((1000,1)),np.zeros((1000,1))))
print(x.shape)
print(y.shape)

model = Sequential()
model.add(Dense(units = 32, input_shape=(10,), activation='relu'))
model.add(Dense(units = 16, activation='relu'))
model.add(Dense(units = 1, activation='sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['binary_accuracy'])
model.fit(x=x,y=y,epochs=2,validation_split=0.2)

print(x.shape)

(2000, 10)
(2000, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                352       
_________________________________________________________________
dense_2 (Dense)              (None, 16)                528       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________
Train on 1600 samples, validate on 400 samples
Epoch 1/2
1600/1600 [==============================] - 3s 2ms/step - loss: 0.5383 - binary_accuracy: 0.8131 - val_loss: 0.1497 - val_binary_accuracy: 1.0000
Epoch 2/2
1600/1600 [==============================] - 1s 676us/step - loss: 0.1832 - binary_accuracy: 1.0000 - val_loss: 0.0309 - val_binary_accuracy: 1.0000
(2000, 10)


#### -------------------------------Load a pb file by tensorflow & Inference using the model

In [12]:
for inp in model.inputs:
    print(inp.op.name)

dense_1_input


In [13]:
for out in model.outputs:
    print(out.op.name)

dense_3/Sigmoid


In [9]:
##load & inference the model ==================
from tensorflow.python.platform import gfile

FILE_PATH = './saved_model/sampleConvertFrozen.pb'

with tf.Session() as sess:
    
    # load model from pb file
    with gfile.FastGFile(FILE_PATH,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    
    # write to tensorboard (check tensorboard for each op names)
#     writer = tf.summary.FileWriter(SAVED_MODEL_DIR+'forzen_log/')
#     writer.add_graph(sess.graph)
#     writer.flush()
#     writer.close()

    
    # print all operation names 
    print('\n===== ouptut operation names =====\n')
    for op in sess.graph.get_operations():
        print(op.name)
    
    # inference by the model (op name must comes with :0 to specify the index of its output)
    tensor_output = sess.graph.get_tensor_by_name('import_1/dense_3/Sigmoid:0')
    tensor_input = sess.graph.get_tensor_by_name('import_1/dense_1_input:0')
    #TODO change get tensor by name to fit to Magma
    predictions = sess.run(tensor_output, feed_dict={tensor_input: x})
    print('\n===== output predicted results =====\n')
    print(predictions)


===== ouptut operation names =====

Adam/iterations/initial_value
Adam/iterations
Adam/iterations/Assign
Adam/iterations/read
Adam/lr/initial_value
Adam/lr
Adam/lr/Assign
Adam/lr/read
Adam/beta_1/initial_value
Adam/beta_1
Adam/beta_1/Assign
Adam/beta_1/read
Adam/beta_2/initial_value
Adam/beta_2
Adam/beta_2/Assign
Adam/beta_2/read
Adam/decay/initial_value
Adam/decay
Adam/decay/Assign
Adam/decay/read
dense_1_input
dense_1/random_uniform/shape
dense_1/random_uniform/min
dense_1/random_uniform/max
dense_1/random_uniform/RandomUniform
dense_1/random_uniform/sub
dense_1/random_uniform/mul
dense_1/random_uniform
dense_1/kernel
dense_1/kernel/Assign
dense_1/kernel/read
dense_1/Const
dense_1/bias
dense_1/bias/Assign
dense_1/bias/read
dense_1/MatMul
dense_1/BiasAdd
dense_1/Relu
dense_2/random_uniform/shape
dense_2/random_uniform/min
dense_2/random_uniform/max
dense_2/random_uniform/RandomUniform
dense_2/random_uniform/sub
dense_2/random_uniform/mul
dense_2/random_uniform
dense_2/kernel
dense_2/

training/Adam/mul_22
training/Adam/add_13
training/Adam/mul_23
training/Adam/sub_15/x
training/Adam/sub_15
training/Adam/Square_4
training/Adam/mul_24
training/Adam/add_14
training/Adam/mul_25
training/Adam/Const_10
training/Adam/Const_11
training/Adam/clip_by_value_5/Minimum
training/Adam/clip_by_value_5
training/Adam/Sqrt_5
training/Adam/add_15/y
training/Adam/add_15
training/Adam/truediv_5
training/Adam/sub_16
training/Adam/Assign_12
training/Adam/Assign_13
training/Adam/Assign_14
training/Adam/mul_26
training/Adam/sub_17/x
training/Adam/sub_17
training/Adam/mul_27
training/Adam/add_16
training/Adam/mul_28
training/Adam/sub_18/x
training/Adam/sub_18
training/Adam/Square_5
training/Adam/mul_29
training/Adam/add_17
training/Adam/mul_30
training/Adam/Const_12
training/Adam/Const_13
training/Adam/clip_by_value_6/Minimum
training/Adam/clip_by_value_6
training/Adam/Sqrt_6
training/Adam/add_18/y
training/Adam/add_18
training/Adam/truediv_6
training/Adam/sub_19
training/Adam/Assign_15
train

KeyError: "The name 'import_1/dense_3/Sigmoid:0' refers to a Tensor which does not exist. The operation, 'import_1/dense_3/Sigmoid', does not exist in the graph."

## d) Inference using TensorRT model

#### d.1) Function to read ".pb" model (TensorRT model is stored in ".pb")

In [11]:
# (can be used to read frozen model or TensorRT model)
def read_pb_graph(model):
    with gfile.FastGFile(model,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    return graph_def

#### prepare data for prediction

In [12]:
images = []
for category in CATEGORIES:
    path_pd = join(PREDICT_DIR, category)
    class_num = CATEGORIES.index(category)
    image_list = os.listdir(path_pd)
    
    for img in image_list:
        images.append([join(path_pd,img),class_num,img])

print(images)
print(images[0][0])


[['./data/predict_dir/0/1202.png', 0, '1202.png'], ['./data/predict_dir/1/5573.png', 1, '5573.png'], ['./data/predict_dir/2/3077.png', 2, '3077.png'], ['./data/predict_dir/3/8448.png', 3, '8448.png'], ['./data/predict_dir/4/8580.png', 4, '8580.png'], ['./data/predict_dir/5/1786.png', 5, '1786.png'], ['./data/predict_dir/6/767.png', 6, '767.png'], ['./data/predict_dir/7/157.png', 7, '157.png']]
./data/predict_dir/0/1202.png


In [13]:
COLS = 5
rows = len(images)//COLS+1

input_img = []

for idx, image in enumerate(images):
    col = idx%COLS
    row = idx//COLS
    
    img = load_img(path=image[0], color_mode='rgb', target_size=DATA_SHAPE)
    # img = img_to_array(img)
    img2predict = img.copy()
    img2predict = img_to_array(img2predict)
    # img2predict = np.expand_dims(img2predict,0)
    img2predict /= 255
    # int(img2predict)
    # print(img2predict.shape)
    input_img.append(img2predict)
    
input_img = np.array(input_img)
print(input_img.shape)

(8, 100, 100, 3)


In [ ]:
## original magma predict
print('-')
COLS = 5
rows = len(images)//COLS+1
print('---')
fig, axes =  plt.subplots(nrows=rows, ncols=COLS, figsize=(10,10), constrained_layout=True)
print('-----')
# delete unused axes
for i in range( -(rows*COLS-len(images)),0):
    fig.delaxes(axes[rows-1,i])
    
for idx, image in enumerate(images):
    col = idx%COLS
    row = idx//COLS
    
    img = load_img(path=image[0], color_mode='rgb', target_size=DATA_SHAPE)
    #img = img_to_array(img)
    img2predict = img.copy()
    img2predict = img_to_array(img2predict)
    img2predict = np.expand_dims(img2predict,0)
    img2predict /= 255
    #int(img2predict)
      
    pred = model.predict(img2predict)
    pred = np.round(pred, 3)
    #pred = np.argmax(pred)
    print(idx, image, pred) 
    img = np.squeeze(img)
    axes[row,col].imshow(img,aspect='equal')
    axes[row,col].set_title('{}\n Prediction: {}\n Real Value: {}'.format(image[2], np.argmax(pred), image[1]),fontsize=16)
    axes[row,col].tick_params(bottom=False, left=False,  labelleft=False, labelbottom=False)

plt.show()

#### d.2) Perform inference using TensorRT model

In [ ]:
TENSORRT_MODEL_PATH = './saved_model/TensorRT_Magma_model_01-07-2019_16-54-31.pb'

print(input_img.shape)

with tf.Session() as sess:
    
    # load model from pb file
    with gfile.FastGFile(TENSORRT_MODEL_PATH,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    
    # write to tensorboard (check tensorboard for each op names)
    writer = tf.summary.FileWriter(SAVED_MODEL_DIR+'forzen_log/')
    writer.add_graph(sess.graph)
    writer.flush()
    writer.close()

    
    # print all operation names 
    print('\n===== ouptut operation names =====\n')
    for op in sess.graph.get_operations():
        print(op.name)
    
    # inference by the model (op name must comes with :0 to specify the index of its output)
    print('\n===== create tensor variables =====\n')
    tensor_output = sess.graph.get_tensor_by_name('import_1/output_tensor/Softmax:0')
    tensor_input = sess.graph.get_tensor_by_name('import_1/input_tensor_input:0')
    
    print('\n===== start prediction =====\n')
    predictions = sess.run(tensor_output, feed_dict={tensor_input: input_img})
    print('\n===== output predicted results =====\n')
    print(predictions)

(8, 100, 100, 3)

===== ouptut operation names =====

Adam/iterations/initial_value
Adam/iterations
Adam/iterations/Assign
Adam/iterations/read
Adam/lr/initial_value
Adam/lr
Adam/lr/Assign
Adam/lr/read
Adam/beta_1/initial_value
Adam/beta_1
Adam/beta_1/Assign
Adam/beta_1/read
Adam/beta_2/initial_value
Adam/beta_2
Adam/beta_2/Assign
Adam/beta_2/read
Adam/decay/initial_value
Adam/decay
Adam/decay/Assign
Adam/decay/read
dense_1_input
dense_1/random_uniform/shape
dense_1/random_uniform/min
dense_1/random_uniform/max
dense_1/random_uniform/RandomUniform
dense_1/random_uniform/sub
dense_1/random_uniform/mul
dense_1/random_uniform
dense_1/kernel
dense_1/kernel/Assign
dense_1/kernel/read
dense_1/Const
dense_1/bias
dense_1/bias/Assign
dense_1/bias/read
dense_1/MatMul
dense_1/BiasAdd
dense_1/Relu
dense_2/random_uniform/shape
dense_2/random_uniform/min
dense_2/random_uniform/max
dense_2/random_uniform/RandomUniform
dense_2/random_uniform/sub
dense_2/random_uniform/mul
dense_2/random_uniform
dense_

In [ ]:
# variable
# print(tr_filename)
# TENSORRT_MODEL_PATH = './saved_model/'+tr_filename

TENSORRT_MODEL_PATH = './saved_model/TensorRT_Magma_model_01-07-2019_16-54-31.pb'

# INPUT_TENSOR = INPUTS + ':0'
# OUTPUT_TENSOR = OUTPUTS + ':0'

graph = tf.Graph()
with graph.as_default():
    with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.50))) as sess:
        # read TensorRT model
        print('read TensorRT model')
        trt_graph = read_pb_graph(TENSORRT_MODEL_PATH)
        
        # print all operation names 
        print('\n===== ouptut operation names =====\n')
        for op in sess.graph.get_operations():
            print(op.name)

        # obtain the corresponding input-output tensor
        print('obtain the corresponding input-output tensor')
        tf.import_graph_def(trt_graph, name='')
        tensor_input = sess.graph.get_tensor_by_name('input_tensor_input:0')
        tensor_output = sess.graph.get_tensor_by_name('output_tensor/Softmax:0')

        # in this case, it demonstrates to perform inference for 50 times
        print('in this case, it demonstrates to perform inference for 50 times')
        total_time = 0; n_time_inference = 3
        print('before Sess run')
        out_pred = sess.run(tensor_output, feed_dict={tensor_input: input_img}) # <--------- boom here
        print('after Sess run')
        for i in range(n_time_inference):
            print('in loop',i)
            t1 = time.time()
            out_pred = sess.run(output, feed_dict={tensor_input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_tensorRT = total_time / n_time_inference
        print("average inference time: ", avg_time_tensorRT)

read TensorRT model

===== ouptut operation names =====

obtain the corresponding input-output tensor
in this case, it demonstrates to perform inference for 50 times
before Sess run


#### d.3) Perform inference using the original tensorflow model

In [ ]:
# variable
FROZEN_MODEL_PATH = './saved_model/Magma_frozen_model.pb'

graph = tf.Graph()
with graph.as_default():
    with tf.Session() as sess:
        # read TensorRT model
        frozen_graph = read_pb_graph(FROZEN_MODEL_PATH)

        # obtain the corresponding input-output tensor
        tf.import_graph_def(frozen_graph, name='')
        input = sess.graph.get_tensor_by_name('conv2d_5_input:0')
        output = sess.graph.get_tensor_by_name('output_tensor_1/Softmax:0')

        # in this case, it demonstrates to perform inference for 50 times
        total_time = 0; n_time_inference = 50
        out_pred = sess.run(output, feed_dict={input: input_img})
        for i in range(n_time_inference):
            t1 = time.time()
            out_pred = sess.run(output, feed_dict={input: input_img})
            t2 = time.time()
            delta_time = t2 - t1
            total_time += delta_time
            print("needed time in inference-" + str(i) + ": ", delta_time)
        avg_time_original_model = total_time / n_time_inference
        print("average inference time: ", avg_time_original_model)
        print("TensorRT improvement compared to the original model:", avg_time_original_model/avg_time_tensorRT)

#### d.4) Plot the prediction result

In [ ]:
for pred in out_pred:
#     plt.figure('img 1')
    plt.imshow(img1, cmap='rgb')
    plt.title('pred:' + str(np.argmax(pred)), fontsize=22)

# Visualize the original and optimized graphs
Using [netron](https://lutzroeder.github.io/netron/), the web application for vitsulaize model graph by upload .pb file.

## 2. Calulate Time, mAP, ACC of Keras model

## 3. Process in TensorRT

## 4. Calculate Time, mAP, ACC of converted model